In these notes, we consider a simple mixed integer linear program (MILP) and derive a Benders decomposition scheme to solve it.

In [152]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using JuMP
using HiGHS

  Activating project at `~/Code/power-systems-optimization`


In [153]:
f = [1, 4];
c = [2, 3];
e = [-2; -3];
A = [1 -3; -1 -3];
E = [1 -2; -1 -1];

The full monolithic model is as follows:

In [154]:
monolithic_model = Model(HiGHS.Optimizer);
@variable(monolithic_model, x[1:2] >= 0, Int);
@variable(monolithic_model, y[1:2] >= 0);
@constraint(monolithic_model, A*x + E*y .<= e);
@objective(monolithic_model,Min, f'*x + c'*y);
latex_formulation(monolithic_model)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2} + x_{1} + 4 x_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
 & x_{1} \in \mathbb{Z}\\
 & x_{2} \in \mathbb{Z}\\
\end{aligned} $$

In [155]:
optimize!(monolithic_model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [1e+00, 4e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 3e+00]
Presolving model
2 rows, 4 cols, 8 nonzeros  0s
2 rows, 4 cols, 8 nonzeros  0s

Solving MIP model with:
   2 rows
   4 cols (0 binary, 2 integer, 0 implied int., 2 continuous)
   8 nonzeros
MIP-Timing:     0.00012 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0    

In [156]:
value.(monolithic_model[:x])

2-element Vector{Float64}:
 0.0
 1.0

Because we are considering a small example problem, the solver was able to quickly compute a solution. However, real-life mixed integer linear programs involve tens of thousands of integer decisions, pushing even the best commercial solvers to their computational limits. 

Note that if we fix the values of integer variables $x_1$ and $x_2$, we obtain a continuous linear program which can be easily solved. For this reason, Benders decomposition is often implemented to solve MILPs, considering integer decisions as complicating variables. 

Let's initialize the master (or planning) problem:

In [157]:
master = Model(HiGHS.Optimizer);
@variable(master, x[1:2] >= 0, Int);
@variable(master,θ>=-1000) #### Initial lower bound on the operational cost (if operational cost is always positive, this can be set to 0)
@objective(master,Min, f'*x + θ);

latex_formulation(master)

$$ \begin{aligned}
\min\quad & x_{1} + 4 x_{2} + θ\\
\text{Subject to} \quad & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & θ \geq -1000\\
 & x_{1} \in \mathbb{Z}\\
 & x_{2} \in \mathbb{Z}\\
\end{aligned} $$

Then, the subproblem is defined including auxiliary variables that act as local copies of the master variables:

In [158]:
subprob = Model(HiGHS.Optimizer);
@variable(subprob, x[1:2] >= 0);
@variable(subprob, y[1:2] >= 0);
@constraint(subprob,A*x + E*y .<= e);
@objective(subprob, Min, c'*y);
latex_formulation(subprob)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
\end{aligned} $$

The Benders decomposition algorithm, starts from solving the master problem:

In [159]:
optimize!(master)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Cost   [1e+00, 4e+00]
  Bound  [1e+03, 1e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -1000           -1000              0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      -1000
  Dual bound        -1000


Using the solution of the master, we define a lower bound to the optimal value of our original MILP:

In [160]:
LB = objective_value(master)

-1000.0

and also guesses for the master variables:

In [161]:
xk = value.(master[:x])

2-element Vector{Float64}:
 0.0
 0.0

We fix these guesses in the subproblem:

In [162]:
fix.(subprob[:x],xk;force=true)
latex_formulation(subprob)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} = 0\\
 & x_{2} = 0\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
\end{aligned} $$

And solve it:

In [163]:
optimize!(subprob)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [2e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 3e+00]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 2(-2); elements 4(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(5) 0s
          2     7.6666666667e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  7.6666666667e+00
Relative P-D gap    :  1.1584935909e-16
HiGHS run time      :          0.00


We can now compute an upper bound to the optimal value of the original MILP as: 

In [164]:
UB = f'*xk + objective_value(subprob)

7.666666666666666

With lower and upper bounds, we can compute the optimality gap, which gives a conserative estimate of the degree of sub-optimality of our current best guess:

In [165]:
gap = (UB-LB)/abs(UB)

131.43478260869566

To improve our guesses, we need to include additional information into the master problem. 

Hence, we derive the so called optimality cuts, which are based on the dual solutions associated with the constraints fixing the values of the master variables in the subproblems:

In [166]:
λ = dual.(FixRef.(subprob[:x]))

2-element Vector{Float64}:
 -2.0
 -8.0

Having only one subproblem, we add one optimality cut per iteration to the master problem:

In [167]:
@constraint(master, master[:θ] >= objective_value(subprob) + λ'*(master[:x]-xk))

2 x[1] + 8 x[2] + θ ≥ 7.666666666666666

We now solve the updated master problem:

In [168]:
latex_formulation(master)

$$ \begin{aligned}
\min\quad & x_{1} + 4 x_{2} + θ\\
\text{Subject to} \quad & 2 x_{1} + 8 x_{2} + θ \geq 7.666666666666666\\
 & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & θ \geq -1000\\
 & x_{1} \in \mathbb{Z}\\
 & x_{2} \in \mathbb{Z}\\
\end{aligned} $$

In [169]:
optimize!(master)

Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [1e+00, 4e+00]
  Bound  [1e+03, 1e+03]
  RHS    [8e+00, 8e+00]
Presolving model
1 rows, 3 cols, 3 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s
1 rows, 2 cols, 2 nonzeros  0s

Solving MIP model with:
   1 rows
   2 cols (0 binary, 1 integer, 0 implied int., 1 continuous)
   2 nonzeros
MIP-Timing:     0.00018 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -1000           inf                 

And obtain new guesses and lower bound:

In [170]:
xk = value.(master[:x])

2-element Vector{Float64}:
 504.0
   0.0

In [171]:
LB = objective_value(master)

-496.0

We then repeat the above steps, fixing the master variable values into the subproblem:

In [172]:
fix.(subprob[:x],xk;force=true)
latex_formulation(subprob)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} = 504\\
 & x_{2} = 0\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
\end{aligned} $$

and solving the subproblem to obtain an upper bound:

In [173]:
optimize!(subprob)

Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [2e+00, 3e+00]
  Bound  [5e+02, 5e+02]
  RHS    [2e+00, 3e+00]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0003367438e+03 Pr: 1(502.667) 0s
          1     7.5900000000e+02 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 1
Objective value     :  7.5900000000e+02
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


In [174]:
UB = min(UB,f'*xk + objective_value(subprob))

7.666666666666666

The current optimality gap is:

In [175]:
gap = (UB-LB)/abs(UB)

65.69565217391305

We have reduced the optimality gap!

Next, we compute a new optimality cut to be added to the master:

In [176]:
λ = dual.(FixRef.(subprob[:x]))

2-element Vector{Float64}:
  1.5
 -4.5

In [177]:
@constraint(master, master[:θ] >= objective_value(subprob) + λ'*(master[:x]-xk))

-1.5 x[1] + 4.5 x[2] + θ ≥ 3

And we solve the updated master problem:

In [178]:
latex_formulation(master)

$$ \begin{aligned}
\min\quad & x_{1} + 4 x_{2} + θ\\
\text{Subject to} \quad & 2 x_{1} + 8 x_{2} + θ \geq 7.666666666666666\\
 & -1.5 x_{1} + 4.5 x_{2} + θ \geq 3\\
 & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & θ \geq -1000\\
 & x_{1} \in \mathbb{Z}\\
 & x_{2} \in \mathbb{Z}\\
\end{aligned} $$

In [179]:
optimize!(master)

Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [1e+00, 4e+00]
  Bound  [1e+03, 1e+03]
  RHS    [3e+00, 8e+00]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s

Solving MIP model with:
   2 rows
   3 cols (0 binary, 2 integer, 0 implied int., 1 continuous)
   6 nonzeros
MIP-Timing:     8.8e-05 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -1000           inf                  inf        0      0      0    

The new lower bound is:

In [180]:
LB = objective_value(master)

-108.0

and solving the subproblem for the new guess gives:

In [181]:
xk = value.(master[:x])
fix.(subprob[:x],xk;force=true)
latex_formulation(subprob)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} = 0\\
 & x_{2} = 223\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
\end{aligned} $$

In [182]:
optimize!(subprob)

Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [2e+00, 3e+00]
  Bound  [2e+02, 2e+02]
  RHS    [2e+00, 3e+00]
Solving LP without presolve, or with basis, or unconstrained
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.0005037380e+03 Pr: 1(333.5) 0s
          1     0.0000000000e+00 Pr: 0(0) 0s
Model status        : Optimal
Simplex   iterations: 1
Objective value     :  0.0000000000e+00
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


In [183]:
UB= min(UB,f'*xk + objective_value(subprob))

7.666666666666666

Resulting in an optimality gap:

In [184]:
gap = (UB-LB)/UB

15.086956521739133

Because the gap is still wide, we keep iterating and compute new cuts to be added to the master:

In [185]:
λ = dual.(FixRef.(subprob[:x]))

2-element Vector{Float64}:
 0.0
 0.0

In [186]:
@constraint(master, master[:θ] >= objective_value(subprob) + λ'*(master[:x]-xk))

θ ≥ 0

We repeat the previous steps once again:

In [187]:
optimize!(master)

Coefficient ranges:
  Matrix [1e+00, 8e+00]
  Cost   [1e+00, 4e+00]
  Bound  [1e+03, 1e+03]
  RHS    [3e+00, 8e+00]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s

Solving MIP model with:
   2 rows
   3 cols (0 binary, 2 integer, 0 implied int., 1 continuous)
   6 nonzeros
MIP-Timing:     0.00011 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0    

In [188]:
LB = objective_value(master)

4.0

In [189]:
xk = value.(master[:x])

2-element Vector{Float64}:
 0.0
 1.0

In [190]:
fix.(subprob[:x],xk;force=true)
latex_formulation(subprob)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} = 0\\
 & x_{2} = 1\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
\end{aligned} $$

In [191]:
optimize!(subprob)

Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [2e+00, 3e+00]
  Bound  [1e+00, 1e+00]
  RHS    [2e+00, 3e+00]
Solving LP without presolve, or with basis, or unconstrained
Model status        : Optimal
Objective value     :  0.0000000000e+00
Relative P-D gap    :  0.0000000000e+00
HiGHS run time      :          0.00


The best upper bound now is:

In [192]:
UB= min(UB,f'*xk + objective_value(subprob))

4.0

Let's check the gap:

In [193]:
gap = (UB-LB)/UB

0.0

Zero optimality gap! We have converged to a solution of our MILP, which coincides with the one obtained solving the monolithic model:

In [194]:
xk

2-element Vector{Float64}:
 0.0
 1.0

In [195]:
value.(monolithic_model[:x])

2-element Vector{Float64}:
 0.0
 1.0

Note that in general the two solutions need not be the same, but they will correspond to the same objective function value.